In [24]:
import math
import numpy as np
import os
import sys
import tensorflow as tf
import pandas as pd

Deeplab demo : https://github.com/tensorflow/models/blob/master/research/deeplab/deeplab_demo.ipynb



In [9]:
PACKAGE_DIR = '/Users/simon.cazals/python/packages'

sys.path.append(os.path.join(PACKAGE_DIR, 'models/research'))

In [10]:
from deeplab.datasets import build_data


In [11]:
image_name = '/Users/simon.cazals/Desktop/projects/steel/data/train_images/58ee62fd7.jpg'

In [52]:
def rle_to_mask(rle_string, height, width):
    if isinstance(rle_string, float) and math.isnan(rle_string):
        return np.zeros((height, width), dtype=np.uint8)
    
    rle_int = list(map(int, rle_string.split(' ')))
    rle_pair = np.array(rle_int).reshape(-1, 2)
    
    img = np.zeros(height * width, dtype=np.uint8)
    for index, length in rle_pair:
        index -= 1
        img[index: index+length] = 1
    
    img = img.reshape(width, height)
    img = img.T
    return img


def masks_to_mask(mask_list):
    assert len(mask_list) == 4, "The list of masks should be of length 4."
    mask = np.zeros(mask_list[0].shape[:2], np.uint8)
    
    for i in range(4):
        mask += (i + 1) * mask_list[i]

    return mask

In [53]:
image_reader = build_data.ImageReader('jpeg', channels=3)
label_reader = build_data.ImageReader('png', channels=1)

image_data = tf.gfile.FastGFile(image_name, 'rb').read()
height, width = image_reader.read_image_dims(image_data)

In [54]:
print(height, width)

256 1600


In [61]:
print(image_data[:100])

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x03\x02\x02\x03\x02\x02\x03\x03\x03\x03\x04\x03\x03\x04\x05\x08\x05\x05\x04\x04\x05\n\x07\x07\x06\x08\x0c\n\x0c\x0c\x0b\n\x0b\x0b\r\x0e\x12\x10\r\x0e\x11\x0e\x0b\x0b\x10\x16\x10\x11\x13\x14\x15\x15\x15\x0c\x0f\x17\x18\x16\x14\x18\x12\x14\x15\x14\xff\xdb\x00C\x01\x03\x04\x04\x05\x04\x05'


In [70]:
from PIL import Image
import io

image = Image.open(image_name)
image_bytes = io.BytesIO()
image.save(image_bytes, format='jpeg')
image_bytes = image_bytes.getvalue()
image_bytes[:100]

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c'

'<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1600x256 at 0x12D0E0D30>'

In [56]:
label_file = '/Users/simon.cazals/Desktop/projects/steel/data/train.csv'
labels_df = pd.read_csv(label_file)
image_class_id_to_rle_mask = dict(zip(labels_df.ImageId_ClassId, labels_df.EncodedPixels))

In [57]:
image_id = image_name.split('/')[-1].split('.')[0]
rle_masks = [image_class_id_to_rle_mask['{}.jpg_{}'.format(image_id, i+1)] for i in range(4)]

In [58]:
masks = [rle_to_mask(rle_mask, height, width) for rle_mask in rle_masks]
mask = masks_to_mask(masks)

In [59]:
for i in range(4):
    print(i, np.max(masks[i]))

0 0
1 0
2 1
3 0


In [60]:
np.max(mask)

3

In [71]:
from PIL import Image
import io

def numpy_to_bytes(image_np, image_format):
    image = Image.fromarray(image_np)
    image_bytes = io.BytesIO()
    image.save(image_bytes, format=image_format)
    return image_bytes.getvalue()

In [ ]:
Image.fromarray(np.uint8(cm.gist_earth(myarray)*255))